In [1]:
from tool_database import tool_database
from transformers import AutoTokenizer

['the', 'quick', 'brown', 'brown', 'fox']


/home/chenjunzhi/.conda/envs/phoenix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("/mntcephfs/lab_data/chenjunzhi/mistralai:Mistral-7B-Instruct-v0.2")
base_path = "/mntcephfs/lab_data/chenjunzhi/data/toolllama_G123_dfs_train.json"
tool_doc_path = "/home/chenjunzhi/JARVIS/easytool/data_toolbench/tool_instruction/toolbench_tool_instruction.json"
tokenizer.add_special_tokens({"bos_token": "<s>", "eos_token": "</s>", "pad_token": "<pad>"})

1

In [3]:
tool_database = tool_database(base_path, tokenizer, tool_doc_path)

In [42]:
query = "I want to do a project on social media influencers, and I want to gather data from Facebook. Can you help me retrieve user information for a specific username, such as 'addidas'? Thank you!\nBegin!"

In [43]:
indices = tool_database.bm25_retrieve_query(query)
indices

[0, 56367, 62805, 163901, 170739]

In [44]:
i, j = tool_database.query_database[56367][0], tool_database.query_database[56367][1]

In [45]:
conversation = tool_database.get_conversations(i)
conversation

[{'from': 'system',
  'value': 'You are AutoGPT, you can use many tools(functions) to do the following task.\nFirst I will give you the task description, and your task start.\nAt each step, you need to give your thought to analyze the status now and what to do next, with a function call to actually excute your step. Your output should follow this format:\nThought:\nAction\nAction Input:\n\nAfter the call, you will get the call result, and you are now in a new state.\nThen you will analyze your status now, then decide what to do next...\nAfter many (Thought-call) pairs, you finally perform the task, then you can give your finial answer.\nRemember: \n1.the state change is irreversible, you can\'t go back to one of the former state, if you want to restart the task, say "I give up and restart".\n2.All the thought is short, at most in 5 sentence.\n3.You can do more then one trys, so if your plan is to continusly try some conditions, you can do one of the conditions per try.\nLet\'s Begin!\n

In [46]:
i, j = tool_database.action_database["userinfo_for_instagram_cheapest"][0]
c = tool_database.get_conversation(i, j)
c

{'from': 'assistant',
 'value': '\nThought: I need to gather user information for a specific Instagram username, \'nike\', as requested by the user. They also want access to the user\'s recent media posts. To achieve this, I will call the \'userinfo_for_instagram_cheapest\' function with the argument of {"username": "nike"}. This will help me retrieve the desired data efficiently and fulfill the user\'s request effectively.\nAction: userinfo_for_instagram_cheapest\nAction Input: {\n  "username": "nike"\n}'}

In [47]:
str(tool_database.tool_doc_database["Cartify.getOrders"])

'{\'name\': \'getOrders\', \'description\': \'The endpoint retrieves all orders.\\n\\nSet the authorization to "Bearer admin" to authorize that the admin can check and confirm all orders.\\n\\nYou should get a JSON object with the following attributes;\\nid (integer): The unique identifier for the order.\\ndeliveryDate(Date): The date that the user wants the order to be delivered\\ndestination(String): Location\\nAlongside with the user who ordered the product\', \'required_parameters\': [], \'optional_parameters\': [], \'Example\': {\'Scenario\': \'If you want to retrieve all orders as an admin\', \'Parameters\': {}}}'

In [30]:
import random
DEFAULT_THOUGHT_TOKEN = "Thought: "
DEFAULT_ACTION_TOKEN = "Action: "
DEFAULT_INPUTS_TOKEN = "Action Input: "
DEFAULT_OBSERVATION_TOKEN = "Observation: "
def bug_retrieve_thought_ICL(database, action, top_k=5):
        thought_ICL = ""
        index = 1
        standard_action = action
        action = action[action.find(".")+1:]
        try:
            action_indices = database.action_database[action]
        except:
            return -1
        if len(action_indices) > top_k:
            random.shuffle(action_indices)
        for action_indice in action_indices:
            i, j = action_indice
            example_conversation = database.get_conversation(i, j)["value"]
            assistant = database.assistant_decompose(example_conversation)
            if assistant["action"] == "Finish":
                continue
            try:
                next_conversation = database.get_conversation(i, j+1)["value"]
            except:
                continue
            try:
                next_assistant = database.assistant_decompose(database.get_conversation(i, j+2)["value"])
                if not isinstance(next_assistant, dict):
                    continue
            except:
                continue
            observation = f"\n{DEFAULT_OBSERVATION_TOKEN}{next_conversation}"
            example_conversation = example_conversation+observation
            next_thought = next_assistant["thought"]
            next_thought = f"{DEFAULT_THOUGHT_TOKEN}{next_thought}"
            example_messages = [{'role': 'user', 'content': example_conversation},
                                {'role': 'assistant', 'content': next_thought}]
            example_ICL = database.get_prompt(example_messages)
            example_ICL = f"{index}. {example_ICL}\n"
            thought_ICL += example_ICL
            index += 1
            if index > top_k:
                break
        
        return thought_ICL

In [48]:
ICL = tool_database.retrieve_thought_ICL("racecards_for_greyhound_racing_uk", top_k=2)
ICL

TypeError: list indices must be integers or slices, not tuple

In [38]:
a = [1,2,3]
random.shuffle(a)
a

[1, 3, 2]